In [1]:
import pickle
import pandas as pd
import numpy as np
import utilities
import preprocess
import parameters

from sklearn.metrics import accuracy_score, hamming_loss
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sentence_transformers import util, SentenceTransformer
from sklearn.metrics import hamming_loss, accuracy_score, f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
import warnings
warnings.filterwarnings("ignore")

## parameters

In [3]:
# importing algorithm parameters
sim_type = parameters.sim_type
random_state = parameters.random_state
test_size = parameters.test_size

# doe
balance_ratio = parameters.balance_ratio
sim_calculation_type = parameters.sim_calculation_type

success_metric = parameters.success_metric
embedding_method = parameters.embedding_method
data_paths = parameters.data_paths
X_num_paths = parameters.X_num_paths
unlabeled_ratios = parameters.unlabeled_ratios

np.random.seed(random_state)

In [4]:
# classifier_object = LinearSVC(class_weight='balanced')
classifier_object = LogisticRegression(class_weight='balanced')

In [5]:
def split_data(X, y):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    X_labeled, X_unlabeled, y_labeled, y_unlabeled = train_test_split(X_train, y_train, test_size=unlabeled_ratios[data], 
                                                                  random_state=random_state)
    
    return X_labeled, y_labeled, X_unlabeled, y_unlabeled, X_test, y_test

In [6]:
def split_data_KFold(X, cv):
    
    from sklearn.model_selection import KFold
    kf = KFold(n_splits=cv, random_state=random_state, shuffle=True)
    
    splits = []
    for train_idx, test_idx in kf.split(X.index):
        
        labeled_idx, unlabeled_idx = train_test_split(train_idx, test_size=unlabeled_ratios[data], random_state=random_state)
        
        splits.append((labeled_idx, unlabeled_idx, test_idx))
        
    return splits    

In [7]:
def read_data(data):
    # reading data
    df = utilities.read_data(data_paths[data])
    # X = df['text'].apply(preprocess.preprocess_text)
    y = df.drop(['text'], axis=1)
    # ------------------------------------------------------------------------------------------------------------------------------
    # reading from a pickle instead of applying vectorization
    # X_num = utilities.vectorize_data(X, embedding_method)
    # X_num = pd.Series([np.squeeze(i) for i in X_num])
    X = pd.read_pickle(X_num_paths[data])
    
    return X, y

# main

In [8]:
def main(data, balance_ratio, sim_calculation_type, single_metric, oversampler_version, batch_size, n_iter=None):
    
    print('*'*100)
    print('\x1b[1;31m'+data+'\x1b[0m')
    
    X, y = read_data(data)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    X_labeled, X_unlabeled, y_labeled, y_unlabeled = train_test_split(X_train, y_train, test_size=unlabeled_ratios[data], 
                                                                  random_state=random_state)
    
    shape_before = X_labeled.shape[0]
    print(X_labeled.shape, X_unlabeled.shape, X_test.shape)
    s_metric = utilities.multilabel_classifier(np.vstack(X_labeled), y_labeled, np.vstack(X_test), y_test, 
                                               success_metric=success_metric,
                                               classifier_object = classifier_object, 
                                               print_results=True)
    # -----------------------------------------------------------------------------------------------------------------------------
    # calculation number of instances to balance dataset
    num_of_new_instances = utilities.calculate_balancing_num_instance_multiclass(y_labeled, balance_ratio, 
                                                                                 calculation_type='metric_based', 
                                                                                 s_metrics=s_metric)
    # -----------------------------------------------------------------------------------------------------------------------------
    # oversampling dataset using unlabeled data with the given ratios
    # print('num_of_new_instances : ',num_of_new_instances)
    if oversampler_version == 'v1':
        validation, X_labeled, y_labeled, X_unlabeled, y_unlabeled = utilities.oversample_dataset_v1(
                                                                        num_of_new_instances, X_labeled, y_labeled, 
                                                                        X_unlabeled, y_unlabeled, X_test, y_test, 
                                                                        sim_calculation_type=sim_calculation_type,
                                                                        batch_size=batch_size)
    elif oversampler_version == 'v2':
        validation, X_labeled, y_labeled, X_unlabeled, y_unlabeled = utilities.oversample_dataset_v2(
                                                                        num_of_new_instances, X_labeled, y_labeled, 
                                                                        X_unlabeled, y_unlabeled, X_test, y_test, 
                                                                        sim_calculation_type=sim_calculation_type,
                                                                        batch_size=batch_size)
    elif oversampler_version == 'v3':
        validation, X_labeled, y_labeled, X_unlabeled, y_unlabeled = utilities.oversample_dataset_v3(
                                                                        num_of_new_instances, X_labeled, y_labeled, 
                                                                        X_unlabeled, y_unlabeled, X_test, y_test, 
                                                                        sim_calculation_type=sim_calculation_type,
                                                                        batch_size=batch_size, 
                                                                        n_iter=n_iter, 
                                                                        single_score=single_metric)
    elif oversampler_version == 'v4':
        validation, X_labeled, y_labeled, X_unlabeled, y_unlabeled, metric_history = utilities.oversample_dataset_v4(
                                                                                         num_of_new_instances, 
                                                                                         X_labeled, y_labeled, 
                                                                                         X_unlabeled, y_unlabeled, 
                                                                                         X_test, y_test, 
                                                                                         sim_calculation_type=sim_calculation_type, 
                                                                                         batch_size=batch_size, 
                                                                                         n_iter=n_iter,
                                                                                         balance_ratio=balance_ratio,
                                                                                         success_metric=success_metric,
                                                                                         single_score=single_metric)
    # -----------------------------------------------------------------------------------------------------------------------------
    # check if the result gets better
    shape_after = X_labeled.shape[0]
    s_metric = utilities.multilabel_classifier(np.vstack(X_labeled), y_labeled, np.vstack(X_test), y_test, 
                                               success_metric=success_metric,
                                               classifier_object = classifier_object, 
                                               print_results=True)
    # comparing the found labels and ground truth
    y_true, y_pred = [], []
    for _, _, _, y_t, y_p in validation:
        y_true.append(list(y_t.values))
        y_pred.append(list(y_p.values()))
    
    acc = 1-hamming_loss(y_true, y_pred)
    emr = accuracy_score(y_true, y_pred)  
    print('-'*30)
    print(f'Shape: before {shape_before}, after {shape_after} : {shape_after-shape_before} instances added...')
    print(f'Exact match ratio : {emr:.2f} ')
    print(f'Accuracy          : {acc:.2f} ')
    print(classification_report(y_true, y_pred))
    print('-'*30)
    
    if oversampler_version == 'v4':
        return metric_history

In [9]:
def run_CV(data, balance_ratio, sim_calculation_type, single_metric, oversampler_version, batch_size, n_iter=None):
    
    CV_results = []
    
    X, y = read_data(data)

    splits = split_data_KFold(X, 5)

    for labeled_idx, unlabeled_idx, test_idx in splits:

        X_labeled = X.loc[labeled_idx]
        y_labeled = y.loc[labeled_idx]
        X_unlabeled = X.loc[unlabeled_idx]
        y_unlabeled = y.loc[unlabeled_idx]
        X_test = X.loc[test_idx]
        y_test = y.loc[test_idx]
    
    
        shape_before = X_labeled.shape[0]

        s_metric_before, initial_scores = utilities.multilabel_classifier(np.vstack(X_labeled), y_labeled, np.vstack(X_test), y_test, 
                                                   success_metric=success_metric,
                                                   classifier_object = classifier_object, 
                                                   print_results=True, return_scores=True)
        # -----------------------------------------------------------------------------------------------------------------------------
        # calculation number of instances to balance dataset
        num_of_new_instances = utilities.calculate_balancing_num_instance_multiclass(y_labeled, balance_ratio, 
                                                                                     calculation_type='metric_based', 
                                                                                     s_metrics=s_metric)
        # -----------------------------------------------------------------------------------------------------------------------------
        # oversampling dataset using unlabeled data with the given ratios
        # print('num_of_new_instances : ',num_of_new_instances)
        if oversampler_version == 'v1':
            validation, X_labeled, y_labeled, X_unlabeled, y_unlabeled = utilities.oversample_dataset_v1(
                                                                            num_of_new_instances, X_labeled, y_labeled, 
                                                                            X_unlabeled, y_unlabeled, X_test, y_test, 
                                                                            sim_calculation_type=sim_calculation_type,
                                                                            batch_size=batch_size)
        elif oversampler_version == 'v2':
            validation, X_labeled, y_labeled, X_unlabeled, y_unlabeled = utilities.oversample_dataset_v2(
                                                                            num_of_new_instances, X_labeled, y_labeled, 
                                                                            X_unlabeled, y_unlabeled, X_test, y_test, 
                                                                            sim_calculation_type=sim_calculation_type,
                                                                            batch_size=batch_size)
        elif oversampler_version == 'v3':
            validation, X_labeled, y_labeled, X_unlabeled, y_unlabeled = utilities.oversample_dataset_v3(
                                                                            num_of_new_instances, X_labeled, y_labeled, 
                                                                            X_unlabeled, y_unlabeled, X_test, y_test, 
                                                                            sim_calculation_type=sim_calculation_type,
                                                                            batch_size=batch_size, 
                                                                            n_iter=n_iter, 
                                                                            single_score=single_metric)
        elif oversampler_version == 'v4':
            validation, X_labeled, y_labeled, X_unlabeled, y_unlabeled, metric_history = utilities.oversample_dataset_v4(
                                                                                             num_of_new_instances, 
                                                                                             X_labeled, y_labeled, 
                                                                                             X_unlabeled, y_unlabeled, 
                                                                                             X_test, y_test, 
                                                                                             sim_calculation_type=sim_calculation_type, 
                                                                                             batch_size=batch_size, 
                                                                                             n_iter=n_iter,
                                                                                             balance_ratio=balance_ratio,
                                                                                             success_metric=success_metric,
                                                                                             single_score=single_metric)
        # -----------------------------------------------------------------------------------------------------------------------------
        # check if the result gets better
        shape_after = X_labeled.shape[0]
        s_metric_after, final_scores = utilities.multilabel_classifier(np.vstack(X_labeled), y_labeled, np.vstack(X_test), y_test, 
                                                   success_metric=success_metric,
                                                   classifier_object = classifier_object, 
                                                   print_results=True, return_scores=True)
        # comparing the found labels and ground truth
        y_true, y_pred = [], []
        for _, _, _, y_t, y_p in validation:
            y_true.append(list(y_t.values))
            y_pred.append(list(y_p.values()))

        acc = 1-hamming_loss(y_true, y_pred)
        emr = accuracy_score(y_true, y_pred)  
        print('-'*30)
        print(f'Shape: before {shape_before}, after {shape_after} : {shape_after-shape_before} instances added...')
        print(f'Exact match ratio : {emr:.2f} ')
        print(f'Accuracy          : {acc:.2f} ')
        print(classification_report(y_true, y_pred))
        print('-'*30)
    
        
        CV_results.append({'shape_before':shape_before,'shape_after':shape_after, 'val_accuracy':acc, 'val_exact_match':emr, 
                          's_metric_before':s_metric_before, 's_metric_after':s_metric_after, 'initial_scores':initial_scores,
                          'final_scores':final_scores, 'validation':validation, })
        
        
    return CV_results

In [10]:
def main_CV(data, balance_ratio, sim_calculation_type, single_metric, oversampler_version, batch_size, n_iter):
    
    print('*'*100)
    print('\x1b[1;31m'+data+'\x1b[0m')
    
    CV_results = run_CV(data, balance_ratio, sim_calculation_type, single_metric, oversampler_version, batch_size, n_iter=None)
    
    return CV_results

In [11]:
data = 'opp115'
balance_ratio = 0.5
sim_calculation_type = 'average'
single_metric = 'f1_score'
batch_size = 1
n_iter = 200

metric_history_v1 = main(data, balance_ratio, sim_calculation_type, single_metric, 'v1', batch_size, n_iter)

metric_history_v2 = main(data, balance_ratio, sim_calculation_type, single_metric, 'v2', batch_size, n_iter)

metric_history_v3 = main(data, balance_ratio, sim_calculation_type, single_metric, 'v3', batch_size, n_iter)

In [12]:
metric_history_v4 = main(data, balance_ratio, sim_calculation_type, single_metric, 'v4', batch_size, 1000)

****************************************************************************************************
opp115
(135,) (2584,) (680,)
Multilabel Classifier Results
LogisticRegression
--------------------
Hamming Loss
Training : 0.05
Test     : 0.08
Exact Match Ratio
Training : 0.55
Test     : 0.39
Macro F1-Score
Training : 0.74
Test     : 0.64
Coverage Error
Training : 1.46
Test     : 2.06
Ranking Loss Error
Training : 0.02
Test     : 0.07
--------------------
Classification Report
                                      precision    recall  f1-score   support

                      Data Retention       0.35      0.74      0.47        19
                       Data Security       0.70      0.82      0.76        51
                        Do Not Track       0.65      0.92      0.76        12
          First Party Collection/Use       0.67      0.72      0.70       218
International and Specific Audiences       0.73      0.84      0.78        55
                Introductory/Generic       0.44 

{'Data Retention': 0.11130587204206836, 'Data Security': 0.10955302366345311, 'Do Not Track': 0.11481156879929887, 'First Party Collection/Use': 0.010517090271691499, 'International and Specific Audiences': 0.06573181419807186, 'Introductory/Generic': 0.03593339176161262, 'Policy Change': 0.10955302366345311, 'Practice not covered': 0.11481156879929887, 'Privacy contact information': 0.10604732690622261, 'Third Party Sharing/Collection': 0.015775635407537247, 'User Access, Edit and Deletion': 0.10955302366345311, 'User Choice/Control': 0.09640666082383874}
Privacy contact information
{'Data Retention': 0.11130587204206836, 'Data Security': 0.10955302366345311, 'Do Not Track': 0.11481156879929887, 'First Party Collection/Use': 0.010517090271691499, 'International and Specific Audiences': 0.06573181419807186, 'Introductory/Generic': 0.03593339176161262, 'Policy Change': 0.10955302366345311, 'Practice not covered': 0.11481156879929887, 'Privacy contact information': 0.10604732690622261, '

{'Data Retention': 0.11159546643417612, 'Data Security': 0.10985178727114212, 'Do Not Track': 0.11508282476024412, 'First Party Collection/Use': 0.010462074978204011, 'International and Specific Audiences': 0.06625980819529206, 'Introductory/Generic': 0.03312990409764603, 'Policy Change': 0.10985178727114212, 'Practice not covered': 0.11508282476024412, 'Privacy contact information': 0.10636442894507411, 'Third Party Sharing/Collection': 0.015693112467306015, 'User Access, Edit and Deletion': 0.10985178727114212, 'User Choice/Control': 0.0967741935483871}
Practice not covered
{'Data Retention': 0.11159546643417612, 'Data Security': 0.10985178727114212, 'Do Not Track': 0.11508282476024412, 'First Party Collection/Use': 0.010462074978204011, 'International and Specific Audiences': 0.06625980819529206, 'Introductory/Generic': 0.03312990409764603, 'Policy Change': 0.10985178727114212, 'Practice not covered': 0.11508282476024412, 'Privacy contact information': 0.10636442894507411, 'Third Pa

{'Data Retention': 0.12090909090909091, 'Data Security': 0.1190909090909091, 'Do Not Track': 0.12454545454545454, 'First Party Collection/Use': 0.012727272727272728, 'International and Specific Audiences': 0.07181818181818182, 'Introductory/Generic': 0.03727272727272727, 'Policy Change': 0.11727272727272728, 'Practice not covered': 0.12454545454545454, 'Privacy contact information': 0.05, 'Third Party Sharing/Collection': 0.013636363636363636, 'User Access, Edit and Deletion': 0.11727272727272728, 'User Choice/Control': 0.09090909090909091}
Practice not covered
{'Data Retention': 0.12090909090909091, 'Data Security': 0.1190909090909091, 'Do Not Track': 0.12454545454545454, 'First Party Collection/Use': 0.012727272727272728, 'International and Specific Audiences': 0.07181818181818182, 'Introductory/Generic': 0.03727272727272727, 'Policy Change': 0.11727272727272728, 'Practice not covered': 0.12454545454545454, 'Privacy contact information': 0.05, 'Third Party Sharing/Collection': 0.0136

{'Data Retention': 0.13631937682570594, 'Data Security': 0.05842259006815969, 'Do Not Track': 0.14021421616358326, 'First Party Collection/Use': 0.01557935735150925, 'International and Specific Audiences': 0.08179162609542356, 'Introductory/Generic': 0.042843232716650435, 'Policy Change': 0.13242453748782862, 'Practice not covered': 0.14021421616358326, 'Privacy contact information': 0.03992210321324245, 'Third Party Sharing/Collection': 0.010710808179162609, 'User Access, Edit and Deletion': 0.13242453748782862, 'User Choice/Control': 0.0691333982473223}
Privacy contact information
{'Data Retention': 0.13631937682570594, 'Data Security': 0.05842259006815969, 'Do Not Track': 0.14021421616358326, 'First Party Collection/Use': 0.01557935735150925, 'International and Specific Audiences': 0.08179162609542356, 'Introductory/Generic': 0.042843232716650435, 'Policy Change': 0.13242453748782862, 'Practice not covered': 0.14021421616358326, 'Privacy contact information': 0.03992210321324245, 'T

...

{'Data Retention': 0.16230366492146597, 'Data Security': 0.0387434554973822, 'Do Not Track': 0.16858638743455498, 'First Party Collection/Use': 0.02198952879581152, 'International and Specific Audiences': 0.09633507853403141, 'Introductory/Generic': 0.050261780104712044, 'Policy Change': 0.031413612565445025, 'Practice not covered': 0.16858638743455498, 'Privacy contact information': 0.04293193717277487, 'Third Party Sharing/Collection': 0.007329842931937173, 'User Access, Edit and Deletion': 0.1581151832460733, 'User Choice/Control': 0.05340314136125655}
Data Security
{'Data Retention': 0.16230366492146597, 'Data Security': 0.0387434554973822, 'Do Not Track': 0.16858638743455498, 'First Party Collection/Use': 0.02198952879581152, 'International and Specific Audiences': 0.09633507853403141, 'Introductory/Generic': 0.050261780104712044, 'Policy Change': 0.031413612565445025, 'Practice not covered': 0.16858638743455498, 'Privacy contact information': 0.04293193717277487, 'Third Party Sha

{'Data Retention': 0.16230366492146597, 'Data Security': 0.0387434554973822, 'Do Not Track': 0.16858638743455498, 'First Party Collection/Use': 0.02198952879581152, 'International and Specific Audiences': 0.09633507853403141, 'Introductory/Generic': 0.050261780104712044, 'Policy Change': 0.031413612565445025, 'Practice not covered': 0.16858638743455498, 'Privacy contact information': 0.04293193717277487, 'Third Party Sharing/Collection': 0.007329842931937173, 'User Access, Edit and Deletion': 0.1581151832460733, 'User Choice/Control': 0.05340314136125655}
Data Retention
{'Data Retention': 0.16230366492146597, 'Data Security': 0.0387434554973822, 'Do Not Track': 0.16858638743455498, 'First Party Collection/Use': 0.02198952879581152, 'International and Specific Audiences': 0.09633507853403141, 'Introductory/Generic': 0.050261780104712044, 'Policy Change': 0.031413612565445025, 'Practice not covered': 0.16858638743455498, 'Privacy contact information': 0.04293193717277487, 'Third Party Sh

{'Data Retention': 0.16230366492146597, 'Data Security': 0.0387434554973822, 'Do Not Track': 0.16858638743455498, 'First Party Collection/Use': 0.02198952879581152, 'International and Specific Audiences': 0.09633507853403141, 'Introductory/Generic': 0.050261780104712044, 'Policy Change': 0.031413612565445025, 'Practice not covered': 0.16858638743455498, 'Privacy contact information': 0.04293193717277487, 'Third Party Sharing/Collection': 0.007329842931937173, 'User Access, Edit and Deletion': 0.1581151832460733, 'User Choice/Control': 0.05340314136125655}
Data Retention
{'Data Retention': 0.16230366492146597, 'Data Security': 0.0387434554973822, 'Do Not Track': 0.16858638743455498, 'First Party Collection/Use': 0.02198952879581152, 'International and Specific Audiences': 0.09633507853403141, 'Introductory/Generic': 0.050261780104712044, 'Policy Change': 0.031413612565445025, 'Practice not covered': 0.16858638743455498, 'Privacy contact information': 0.04293193717277487, 'Third Party Sh

{'Data Retention': 0.16230366492146597, 'Data Security': 0.0387434554973822, 'Do Not Track': 0.16858638743455498, 'First Party Collection/Use': 0.02198952879581152, 'International and Specific Audiences': 0.09633507853403141, 'Introductory/Generic': 0.050261780104712044, 'Policy Change': 0.031413612565445025, 'Practice not covered': 0.16858638743455498, 'Privacy contact information': 0.04293193717277487, 'Third Party Sharing/Collection': 0.007329842931937173, 'User Access, Edit and Deletion': 0.1581151832460733, 'User Choice/Control': 0.05340314136125655}
Privacy contact information
{'Data Retention': 0.16230366492146597, 'Data Security': 0.0387434554973822, 'Do Not Track': 0.16858638743455498, 'First Party Collection/Use': 0.02198952879581152, 'International and Specific Audiences': 0.09633507853403141, 'Introductory/Generic': 0.050261780104712044, 'Policy Change': 0.031413612565445025, 'Practice not covered': 0.16858638743455498, 'Privacy contact information': 0.04293193717277487, 'T

{'Data Retention': 0.16230366492146597, 'Data Security': 0.0387434554973822, 'Do Not Track': 0.16858638743455498, 'First Party Collection/Use': 0.02198952879581152, 'International and Specific Audiences': 0.09633507853403141, 'Introductory/Generic': 0.050261780104712044, 'Policy Change': 0.031413612565445025, 'Practice not covered': 0.16858638743455498, 'Privacy contact information': 0.04293193717277487, 'Third Party Sharing/Collection': 0.007329842931937173, 'User Access, Edit and Deletion': 0.1581151832460733, 'User Choice/Control': 0.05340314136125655}
Do Not Track
{'Data Retention': 0.16230366492146597, 'Data Security': 0.0387434554973822, 'Do Not Track': 0.16858638743455498, 'First Party Collection/Use': 0.02198952879581152, 'International and Specific Audiences': 0.09633507853403141, 'Introductory/Generic': 0.050261780104712044, 'Policy Change': 0.031413612565445025, 'Practice not covered': 0.16858638743455498, 'Privacy contact information': 0.04293193717277487, 'Third Party Shar

In [13]:
Shape: before 135, after 190 : 55 instances added...

SyntaxError: invalid syntax (995201890.py, line 1)

In [ ]:
add dimensionality reduction

In [ ]:
stop

In [ ]:
data = 'opp115'
balance_ratio = 0.5
sim_calculation_type = 'average'
single_metric = 'f1_score'
oversampler_version = 'v4'
batch_size = 1
n_iter = 500

In [ ]:
main(data, balance_ratio, sim_calculation_type, single_metric, oversampler_version, batch_size, n_iter)

In [ ]:
main(data, balance_ratio, sim_calculation_type, single_metric, oversampler_version, batch_size, n_iter)

In [ ]:
stop

In [ ]:
main('opp115', balance_ratio, sim_calculation_type, single_metric, 'v1', batch_size)

In [ ]:
main('opp115', balance_ratio, sim_calculation_type,  single_metric, 'v2', batch_size)

In [ ]:
main('opp115', balance_ratio, sim_calculation_type,  'coverage', 'v3', batch_size, n_iter)

In [ ]:
main('opp115', balance_ratio, sim_calculation_type, 'coverage', 'v4', batch_size, n_iter)

In [ ]:
ss

In [ ]:
data = 'opp115'
balance_ratio = 0.5
sim_calculation_type = 'average'
single_metric = 'coverage'
oversampler_version = 'v1'
batch_size = 1
n_iter = 500

In [ ]:
parameters = {
'data' : ['opp115', 'ohsumed', 'reuters'],
'balance_ratio' : [0.2, 0.5],
'sim_calculation_type' : ['average', 'safe_interval'],
'single_metric' : ['accuracy', 'f1_score', 'coverage', 'label_ranking', 'roc_auc_score', 'log_loss', 'average_precision',
                   'brier_loss', 'hamming_loss', 'precision', 'recall', 'zero_one_loss', 'label_ranking_average_precision'],
'oversampler_version' : ['v1', 'v2', 'v3', 'v4'],
'batch_size' : [1,3,5],
'n_iter' : [100, 1000],
}

In [ ]:
replication_size = 5

In [ ]:
for data in parameters['data']:
    for balance_ratio in parameters['balance_ratio']:
        for sim_calculation_type in parameters['sim_calculation_type']:
            for single_metric in parameters['single_metric']:
                for oversampler_version in parameters['oversampler_version']:
                    for batch_size in parameters['batch_size']:
                        for n_iter in parameters['n_iter']:
                            
                            results = []
                            param_list = [data, balance_ratio, sim_calculation_type, single_metric, oversampler_version,
                                         batch_size, n_iter]
                            
                            for i in range(replication_size):
                                results.append(main(data, balance_ratio, sim_calculation_type, single_metric, 
                                                    oversampler_version, batch_size, n_iter))
                            

In [ ]:
num_of_new_instances = {'a':20, 'b':30, 'c':50}
n_iter = 200

In [ ]:
n_iter_dist = {k:int(n_iter*v/sum(num_of_new_instances.values())) for k,v in num_of_new_instances.items()}

In [ ]:
n_iter_dist

In [ ]:
main('opp115', embedding_method, classifier_object, sim_type, sim_calculation_type, success_metric, 'label-ranking')

In [ ]:
asdas

In [ ]:
# oversampling methods

oversample_dataset_v2(num_of_new_instances, X_labeled, y_labeled, X_unlabeled, y_unlabeled, X_test, y_test, sim_calculation_type, batch_size)
oversample_dataset_v3(num_of_new_instances, X_labeled, y_labeled, X_unlabeled, y_unlabeled, X_test, y_test, sim_calculation_type, batch_size, n_iter)
oversample_dataset_v4(num_of_new_instances, X_labeled, y_labeled, X_unlabeled, y_unlabeled, X_test, y_test, sim_calculation_type, batch_size, n_iter, balance_ratio, success_metric)

In [ ]:
for data in data_paths.keys():
    main(data, embedding_method, classifier_object, sim_type, sim_calculation_type, success_metric)

In [ ]:
stop

In [ ]:
import numpy as np

In [ ]:
df = utilities.read_data(data_paths[data])
X = df['text'].apply(preprocess.preprocess_text)
y = df.drop(['text'], axis=1)

In [ ]:
y[y[col] == 1].index